In [ ]:
!pip3 install matplotlib
!pip3 install pandas
!pip3 install sklearn
!pip3 install tensorflow

In [41]:
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping



In [42]:
train_df.head()

,id_code,diagnosis,filepath
3832,CN-2037.png,CN,ADNI_IMAGES/CN/CN-2037.png
7092,AD-2677.png,AD,ADNI_IMAGES/AD/AD-2677.png
6995,AD-3479.png,AD,ADNI_IMAGES/AD/AD-3479.png
1241,CN-3933.png,CN,ADNI_IMAGES/CN/CN-3933.png
8141,AD-2268.png,AD,ADNI_IMAGES/AD/AD-2268.png


In [48]:
df = pd.read_csv("binary_data.csv")

df['id_code'] = df['id_code'].astype(str) + '.png'

X = df["id_code"]
y = df["diagnosis"]


df['filepath'] = df.apply(lambda row: os.path.join("ADNI_IMAGES",
                                                   'AD' if row['diagnosis'] == 4 else 'CN',
                                                   row['id_code']), axis=1)

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['diagnosis'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['diagnosis'], random_state=42)

train_df['diagnosis'] = train_df['diagnosis'].map({0: 'CN', 4: 'AD'})
val_df['diagnosis'] = val_df['diagnosis'].map({0: 'CN', 4: 'AD'})
test_df['diagnosis'] = test_df['diagnosis'].map({0: 'CN', 4: 'AD'})




datagen = ImageDataGenerator(rescale=1./255)


train_data = datagen.flow_from_dataframe(train_df, x_col='filepath', y_col='diagnosis',
                                         target_size=(224, 224), batch_size=32,
                                         class_mode='binary', shuffle=True, classes=['CN', 'AD'],)

val_data = datagen.flow_from_dataframe(val_df, x_col='filepath', y_col='diagnosis',
                                       target_size=(224, 224), batch_size=32,
                                       class_mode='binary', shuffle=False, classes=['CN', 'AD'],)

test_data = datagen.flow_from_dataframe(test_df, x_col='filepath', y_col='diagnosis',
                                        target_size=(224, 224), batch_size=32,
                                        class_mode='binary', shuffle=False, classes=['CN', 'AD'],)

print(train_data.class_indices)



Found 6521 validated image filenames belonging to 2 classes.
Found 815 validated image filenames belonging to 2 classes.
Found 816 validated image filenames belonging to 2 classes.
{'CN': 0, 'AD': 1}


In [51]:
#Base model with more layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Conv2D(64, (3, 3), activation='relu'),
    # BatchNormalization(),
    # MaxPooling2D(pool_size=(2, 2)),

    # Conv2D(128, (3, 3), activation='relu'),
    # BatchNormalization(),
    # MaxPooling2D(pool_size=(2, 2)),

    # Conv2D(256, (3, 3), activation='relu'),
    # BatchNormalization(),
    #MaxPooling2D(pool_size=(2, 2)),

    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # <--- binary output (Alzheimer's vs Normal)
])

early = EarlyStopping(patience=10, restore_best_weights=True)


#model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

learning_rate = 0.001  # Example: lower than the default 0.001

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[early])


#model.fit(
    #train_data,
    #validation_data=val_data,
    #epochs=20
#)


Epoch 1/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 341ms/step - accuracy: 0.4922 - loss: 0.6952 - val_accuracy: 0.4871 - val_loss: 0.6935
Epoch 2/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 69s 338ms/step - accuracy: 0.5007 - loss: 0.6936 - val_accuracy: 0.4945 - val_loss: 0.6937
Epoch 3/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 342ms/step - accuracy: 0.4942 - loss: 0.6943 - val_accuracy: 0.4969 - val_loss: 0.6945
Epoch 4/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 345ms/step - accuracy: 0.5046 - loss: 0.6936 - val_accuracy: 0.5202 - val_loss: 0.6929
Epoch 5/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 71s 348ms/step - accuracy: 0.5034 - loss: 0.6937 - val_accuracy: 0.5006 - val_loss: 0.6934
Epoch 6/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 73s 359ms/step - accuracy: 0.5069 - loss: 0.6933 - val_accuracy: 0.4945 - val_loss: 0.6931
Epoch 7/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 73s 359ms/step - accuracy: 0.5019 - loss: 0.6934 - val_accuracy: 0.4994 - val_loss: 0.6932
Epoch 8/20
204/204 ━━━━━━━━━━━━━━━━━━━━ 73s 357ms/step - accuracy: 0.4889 - loss: 0

In [ ]:
#Original base model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # <--- binary output (Alzheimer's vs Normal)
])


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20)

In [ ]:
# Transfer learning model with VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = Sequential([
    base_model, 
    GlobalAveragePooling2D(),  
    Dense(128, activation='relu'),  
    #Dropout(0.5),  
    Dense(1, activation='sigmoid')  
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

In [ ]:
# Transfer learning model with ResNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

